In [1]:
# test train task
# train a model with enough runs
# compare results with default MD DFT

In [1]:
# import modules
from pathlib import Path
import torch
import logging
import shutil
from collections import deque
from dataclasses import dataclass
from random import shuffle, sample
from typing import Dict, Any, Optional
import json
from functools import partial, update_wrapper
import numpy as np
import time
import pickle

import ase
from ase.db import connect
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from fff.learning.gc.ase import SchnetCalculator
from fff.learning.gc.functions import GCSchNetForcefield
from fff.learning.gc.models import SchNet, load_pretrained_model
from fff.learning.util.messages import TorchMessage
from fff.sampling.md import MolecularDynamics
from fff.simulation import run_calculator, _run_calculator
from fff.simulation.utils import read_from_string, write_to_string

In [1]:
# path and varaibles
multisite_path = "/home/yxx/work/project/colmena/multisite_"
training_set = multisite_path + \
    "/data/forcefields/starting-model/initial-database.db"
model_path = multisite_path + "/data/forcefields/starting-model/starting-model"
search_path = training_set
out_dir = Path(multisite_path) / f"my_test/temp"
out_dir.mkdir(parents=True, exist_ok=True)

starting_model = torch.load(model_path, map_location='cpu')

num_epochs = 512
huber_deltas = (1, 10)
sampler_kwargs = {'device': "cpu", 'timestep': 0.1, 'log_interval': 10}
sampler = MolecularDynamics()
n_models = 1
n_qc_workers = 8
min_run_length = 200
max_run_length = 2000
energy_tolerance = 0.1

NameError: name 'Path' is not defined

In [ ]:
# train model pretreat

# Apply wrappers to functions that will be used to fix certain requirements
def _wrap(func, **kwargs):
    out = partial(func, **kwargs)
    update_wrapper(out, func)
    return out

# MD objectives


@dataclass
class Trajectory:
    """Tracks the state of searching along individual trajectories

    We mark the starting point, the last point produced from sampling,
    and the last point we produced that has been validated
    """
    id: int  # ID number of the
    starting: ase.Atoms  # Starting point of the trajectory
    current_timestep = 0  # How many timesteps have been used so far
    last_validated: ase.Atoms = None  # Last validated point on the trajectory
    current: ase.Atoms = None  # Last point produced along the trajectory
    last_run_length: int = 0  # How long between current and last_validated
    name: str = None  # Name of the trajectory

    def __post_init__(self):
        self.last_validated = self.current = self.starting

    def update_current_structure(self, strc: ase.Atoms, run_length: int):
        """Update the structure that has yet to be updated

        Args:
            strc: Structure produced by sampling
            run_length: How many timesteps were performed in sampling run
        """
        self.current = strc.copy()
        self.last_run_length = run_length

    def set_validation(self, success: bool):
        """Set whether the trajectory was successfully validated

        Args:
            success: Whether the validation was successful
        """
        if success:
            self.last_validated = self.current  # Move the last validated forward
            self.current_timestep += self.last_run_length


@dataclass
class SimulationTask:
    atoms: ase.Atoms  # Structure to be run
    traj_id: int  # Which trajectory this came from
    ml_eng: float  # Energy predicted from machine learning model
    ml_std: Optional[float] = None  # Uncertainty of the model


# get model
schnet = GCSchNetForcefield(starting_model)

# copy training data
train_path = out_dir / "train.db"
shutil.copyfile(training_set, train_path)

# wrap functions
# train model
my_train_schnet = _wrap(schnet.train, num_epochs=num_epochs, device='cuda',
                        patience=8, reset_weights=False,
                        huber_deltas=huber_deltas)

# evaluate model
my_eval_schnet = _wrap(schnet.evaluate, device='cuda')

# use model sampling
my_run_dynamics = _wrap(sampler.run_sampling, **sampler_kwargs)


# prepare input
# Load in the search space
with connect(search_path) as db:
    search_space = [Trajectory(i, x.toatoms(), name=x.get(
        'filename', f'traj-{i}')) for i, x in enumerate(db.select(''))]
    shuffle(search_space)
    search_space = deque(search_space)

# Load in the training dataset
with connect(train_path) as db:
    all_examples = np.array([x.toatoms() for x in db.select("")], dtype=object)

    # Remove the unrealistic structures
    # if self.max_force is not None:
    #     all_examples = [a for a in all_examples if np.abs(a.get_forces()).max() < max_force]

# search space queue
to_audit: dict[int, Trajectory] = {}  # Trajectories that need to be audited
audit_results: deque[float] = deque(maxlen=50)  # Results of the last 50 audits
task_queue_audit = []

# Prepare the initial model
StartModelMessage = TorchMessage(starting_model)
ActiveModelMessage = SchnetCalculator(starting_model)
# Prepare the dataset
train_sets = []
valid_sets = []
n_train = int(len(all_examples) * 0.9)
for _ in range(n_models):
    shuffle(all_examples)
    train_sets.append(all_examples[:n_train])
    valid_sets.append(all_examples[n_train:])

# store model and log
model_msgs = []
train_logs = []

In [ ]:
# train model
for i in range(0, 1):
    for i, train_set in enumerate(valid_sets):
        model_msg, train_log = my_train_schnet(
            model_msg=StartModelMessage, train_data=train_set, valid_data=valid_sets[i])
        model_msgs.append(model_msg)
        train_logs.append(train_log)

    # store model
    # now we just test one model
    model_save_path = out_dir / "model.pth"
    with open(model_save_path, 'wb') as fp:
        torch.save(model_msgs[0].get_model(), fp)
    # Save the training data
    with open(out_dir / 'training-history.json', 'a') as fp:
        print(json.dumps(train_logs[0].to_dict(orient='list')), file=fp)

    active_model_proxy = SchnetCalculator(model_msgs[0].get_model())
    StartModelMessage = TorchMessage(model_msgs[0].get_model())
    model_msgs = []
    train_logs = []

In [ ]:
# use model for sampling
# sampling tasks loop
for i in range(0, 100):
    # Pick the next eligible trajectory and start from the last validated structure
    trajectory = search_space.popleft()
    starting_point = trajectory.starting

    # Initialize the structure if need be
    if trajectory.current_timestep == 0:
        MaxwellBoltzmannDistribution(starting_point, temperature_K=100)
        print('Initialized temperature to 100K')
    # Add the structure to a list of those being validated
    to_audit[trajectory.id] = trajectory

    # Determine the run length based on observations of errors
    run_length = min_run_length
    if len(audit_results) > n_qc_workers:
        # Predict run length given audit error
        error_per_step = np.median(audit_results)
        target_error = energy_tolerance * 2
        estimated_run_length = int(target_error / error_per_step)
        print(
            f'Estimated run length of {estimated_run_length} steps to have an error of {target_error:.3f} eV/atom')
        # Keep to within the user-defined bounds
        run_length = max(min_run_length, min(
            max_run_length, estimated_run_length))

    # do sampling
    audit, traj = my_run_dynamics(
        atoms=starting_point, steps=run_length, calc=active_model_proxy)
    # print(audit)
    # print(len(traj))
    # add to list
    to_audit[trajectory.id].update_current_structure(audit, run_length)
    task_queue_audit.append(SimulationTask(
        atoms=traj[-1], traj_id=trajectory.id, ml_eng=traj[-1].get_potential_energy()))

print(len(task_queue_audit))

Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initialized temperature to 100K
Initiali

In [ ]:
# store sampling result
import pickle
with open(out_dir / 'task_queue_audit', 'wb') as fp:
    pickle.dump(task_queue_audit, fp)

In [ ]:
# test simulation
# real MD simulation
tempdir = "./temp"

# set calculator
calc = dict(calc='psi4', method='pbe0-d3', basis='aug-cc-pvdz', num_threads=24)
my_run_simulation = _wrap(run_calculator, calc=calc, temp_path=tempdir)

# prepare input
to_run = task_queue_audit[-1]
ml_eng = to_run.ml_eng
atoms = to_run.atoms
atoms.set_center_of_mass([0, 0, 0])
xyz = write_to_string(atoms, 'xyz')

# run
value = my_run_simulation(xyz)

# result
atoms = read_from_string(value, 'json')
dft_energy = atoms.get_potential_energy()
diff_energy = abs(dft_energy - ml_eng) / len(atoms)
print(diff_energy)

  Threads set to 24 by Python driver.
  Threads set to 24 by Python driver.
0.07407622481029345


In [ ]:
# simple grid search
import time
import itertools
from ase.build import molecule
from concurrent.futures import ProcessPoolExecutor, as_completed
from fff.simulation import run_calculator, _run_calculator
from fff.simulation.utils import read_from_string, write_to_string

parameter_space = []
tempdir = "./temp"

def generate_search_space(num_cpus, max_parallelism):
    search_space = []
    for parallelism in range(1, max_parallelism + 1):
        combinations = itertools.combinations_with_replacement(
            range(1, num_cpus + 1), parallelism)
        for combination in combinations:
            if sum(combination) == num_cpus:
                search_space.append(combination)
    return search_space


def bundle_simulation_task(run_parames=[8, 8, 8], atoms_queue=[]):
    # simulation here
    atoms = []
    task_infos = []
    # for _ in range(0,len(run_parames)):
    #     ##TODO we should choose proper atoms here
    #     atoms.append(atoms_queue.pop())
    
    # simple test
    # atoms.append(molecule('H2O'))
    atoms = molecule('H2O')
    with ProcessPoolExecutor(max_workers=len(run_parames)) as exe:
        start_times = {}
        futs = []
        for i,cpus in enumerate(run_parames):
            # atoms[0].set_center_of_mass([0, 0, 0])
            xyz = write_to_string(atoms, 'xyz')
            calc = dict(calc='psi4', method='pbe0-d3', basis='aug-cc-pvdz', num_threads=cpus)            
            
            fut = exe.submit(_run_calculator, str(xyz), calc, tempdir)
            start_times[fut] = time.time()
            futs.append(fut)
            
    execution_times = []
    for fut in as_completed(futs):
        execution_times.append(time.time() - start_times[fut])
        calculated = read_from_string(fut.result(), 'json')
        dft_energy = calculated.get_potential_energy()
        # diff_energy = abs(dft_energy - ml_eng) / len(calculated)

    # task_info = {
    #     'atoms': calculated,
    #     'run_parames': cpus,
    #     'time': end - start,
    #     #TODO 'ml_eng' : to_run.ml_eng,
    #     'dft_eng': dft_energy,
    # }
    # task_infos.append(task_info)
    return execution_times


num_cpus = 24
max_parallelism = 8
parameter_space = generate_search_space(num_cpus, max_parallelism)

print(len(parameter_space))
print(parameter_space)

gird_results = []
for run_parames in parameter_space:
    gird_results.append((run_parames,bundle_simulation_task(run_parames)))


In [ ]:
import json
print(len(gird_results))

with open('./temp/grid_results.json', 'w') as json_file:
    json.dump(gird_results, json_file)

821


In [ ]:
# test pickle serialization

with open(out_dir / 'task_queue_audit', 'rb') as fp:
    task_queue_audit_test = pickle.load(fp)

# test simulation
# real MD simulation
tempdir = "./temp"

# set calculator
calc = dict(calc='psi4', method='pbe0-d3', basis='aug-cc-pvdz', num_threads=24)
my_run_simulation = _wrap(run_calculator, calc=calc, temp_path=tempdir)

# prepare input

# to_run = task_queue_audit_test[-1]
# ml_eng = to_run.ml_eng
# atoms = to_run.atoms
# atoms.set_center_of_mass([0, 0, 0])

# atoms = molecule('H2O')
# xyz = write_to_string(atoms, 'xyz')

# run
# value = my_run_simulation(xyz)

# result
# atoms = read_from_string(value, 'json')
# dft_energy = atoms.get_potential_energy()
# diff_energy = abs(dft_energy - ml_eng) / len(atoms)
# print(diff_energy)


execution_times = []
diff_energies = []
simulation_lists = []
for to_run in task_queue_audit_test:
    start = time.time()
    ml_eng = to_run.ml_eng
    atoms = to_run.atoms
    atoms.set_center_of_mass([0, 0, 0])
    xyz = write_to_string(atoms, 'xyz')

    # run
    value = my_run_simulation(xyz)
    
    # result
    atoms = read_from_string(value, 'json')
    dft_energy = atoms.get_potential_energy()
    diff_energy = abs(dft_energy - ml_eng) / len(atoms)
    diff_energies.append(diff_energy)
    simulation_lists.append(atoms)
    execution_times.append(time.time() - start)
    # print(diff_energy)

data = []
for x,y,z in simulation_lists,diff_energies,execution_times:
    data.append((x,y,z))
with open('./temp/simulation_tasks_execution_time.json', 'w') as json_file:
    json.dump(data, json_file)

  Threads set to 24 by Python driver.
  Threads set to 24 by Python driver.


ValueError: Calculation failed: 
Fatal Error: Matrix::power: C_DSYEV failed
Error occurred in file: /scratch/psilocaluser/conda-builds/psi4-multiout_1657298395608/work/psi4/src/psi4/libmints/matrix.cc on line: 2330
The most recent 5 function calls were:

psi::FittingMetric::form_eig_inverse(double)
psi::DiskDFJK::preiterations()


In [ ]:
import ase
import sys
sys.path.append('../../multisite_')
# sys.path.append('../my_util')
from evt.evo_sch import *
from my_util.data_structure import *
from fff.simulation import run_calculator,_run_calculator
from fff.simulation.utils import read_from_string, write_to_string
 
